# Extended Kalman filter for Abkowitz model
An Extended Kalman filter with an Abkowitz model as the predictor will be developed.

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import sympy as sp

import vessel_manoeuvring_models.visualization.book_format as book_format
book_format.set_style()
from vessel_manoeuvring_models.substitute_dynamic_symbols import lambdify
from sympy import Matrix
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify
from sympy.physics.vector.printing import vpprint, vlatex
from vessel_manoeuvring_models.data import mdl
from vessel_manoeuvring_models.kalman_filter import extended_kalman_filter
from vessel_manoeuvring_models.models.vmm import ModelSimulator
from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.models import vmm_abkowitz as vmm
import vessel_manoeuvring_models.prime_system as prime_system

In [ ]:
#format the book
import vessel_manoeuvring_models.visualization.book_format as book_format
book_format.set_style()

## Load model

In [ ]:
model = ModelSimulator.load('../models/model_VCT_abkowitz.pkl')

### Run a zigzag simulation with the model

In [ ]:
u0_=2
angle_deg = 35
result = model.zigzag(u0=u0_, angle=angle_deg)

In [ ]:
result.track_plot();
result.plot(compare=False);

In [ ]:
x_0,x_01d = sp.symbols('x_0 \dot{x_0}')
y_0,y_01d = sp.symbols('y_0 \dot{y_0}')
u,u1d = sp.symbols('u \dot{u}')
v,v1d = sp.symbols('v \dot{v}')
delta = sp.symbols('delta')
psi,psi1d = sp.symbols('\Psi \dot{\Psi}')
r,r1d = sp.symbols('r \dot{r}')



In [ ]:
from vessel_manoeuvring_models.symbols import dynamicsymbols
subs = [

    (dynamicsymbols('u').diff(),u1d),
    (dynamicsymbols('v').diff(),v1d),
    (dynamicsymbols('r').diff(),r1d),
    
    (dynamicsymbols('u'),u),
    (dynamicsymbols('v'),v),
    (dynamicsymbols('r'),r),
        
    (dynamicsymbols('delta'),delta),

]

X_eq = vmm.X_eq.subs(subs)
Y_eq = vmm.Y_eq.subs(subs)
N_eq = vmm.N_eq.subs(subs)

In [ ]:
subs_prime = [
    
    (m,m/prime_system.df_prime.mass.denominator),
    (I_z,I_z/prime_system.df_prime.inertia_moment.denominator),
    (x_G,x_G/prime_system.df_prime.length.denominator),
    
    (u, u/sp.sqrt(u**2+v**2)),
    (v, v/sp.sqrt(u**2+v**2)),
    (r, r/(sp.sqrt(u**2+v**2))/L),
    
    (u1d, u1d/((u**2+v**2)/L)),
    (v1d, v1d/((u**2+v**2)/L)),
    (r1d, r1d/((u**2+v**2)/(L**2))),
    
]

In [ ]:
X_eq_SI = X_eq.subs(subs_prime)
Y_eq_SI = Y_eq.subs(subs_prime)
N_eq_SI = N_eq.subs(subs_prime)

In [ ]:
A_SI, b_SI = sp.linear_eq_to_matrix([X_eq_SI, Y_eq_SI, N_eq_SI], [u1d, v1d, r1d])

In [ ]:
A_SI

In [ ]:
A_inv = A_SI.inv()
S = sp.symbols('S')
eq_S=sp.Eq(S,-sp.fraction(A_inv[1,1])[1])

In [ ]:
eq_S

In [ ]:
acceleration = sp.matrices.MutableDenseMatrix([u1d,v1d,r1d])
acceleartion_eq = sp.Eq(acceleration,A_inv * b_SI)

acceleartion_eq

In [ ]:
u1d_function = sp.Function(r'\dot{u}')(u,v,r,delta)
v1d_function = sp.Function(r'\dot{v}')(u,v,r,delta)
r_function = sp.Function(r'\dot{r}')(u,v,r,delta)


eq_u1d_function = sp.Eq(u1d_function, acceleartion_eq.rhs[0])
eq_v1d_function = sp.Eq(v1d_function, acceleartion_eq.rhs[1])
eq_r_function = sp.Eq(r_function, acceleartion_eq.rhs[2])

display(eq_u1d_function)
display(eq_v1d_function)
display(eq_r_function)



In [ ]:
eq_acceleartion_function = sp.Eq(acceleration,sp.matrices.MutableDenseMatrix([u1d_function,v1d_function,r_function]))
display(eq_acceleartion_function)

### Simulation model

In [ ]:
x, x1d = sp.symbols(r'\vec{x} \dot{\vec{x}}')  # State vector
h = sp.symbols('h')
u_input = sp.symbols(r'u_{input}')  # input vector
w_noise = sp.symbols(r'w_{noise}')  # input vector

In [ ]:
f = sp.Function('f')(x,u_input,w_noise)
eq_system = sp.Eq(x1d, f)
eq_system

Where the state vector $\vec{x}$:

In [ ]:
eq_x = sp.Eq(x, sp.UnevaluatedExpr(Matrix([x_0, y_0, psi, u, v, r])))
eq_x

In [ ]:
eq_x0_1d = sp.Eq(x_01d,u)
eq_y0_1d = sp.Eq(y_01d,v)
eq_psi_1d = sp.Eq(psi1d,r)

display(eq_x0_1d)
display(eq_y0_1d)
display(eq_psi_1d)


In [ ]:
eq_f =sp.Eq(f,
sp.UnevaluatedExpr(
Matrix([
    eq_x0_1d.rhs,
    eq_y0_1d.rhs,
    eq_psi_1d.rhs,
    eq_u1d_function.lhs,
    eq_v1d_function.lhs,
    eq_r_function.lhs,
    
]
))
)
display(eq_f)


In [ ]:
f_explicit = Matrix([
    eq_x0_1d.rhs,
    eq_y0_1d.rhs,
    eq_psi_1d.rhs,
    eq_u1d_function.rhs,
    eq_v1d_function.rhs,
    eq_r_function.rhs,
    
]
)


In [ ]:
eq_f.rhs.doit().jacobian(eq_x.rhs.doit())

In [ ]:
Ad = sp.eye(6,6) + Matrix([
    eq_x0_1d.rhs,
    eq_y0_1d.rhs,
    eq_psi_1d.rhs,
    eq_u1d_function.rhs,
    eq_v1d_function.rhs,
    eq_r_function.rhs,
    
]
).jacobian(eq_x.rhs.doit())*h

In [ ]:
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
subs = {value: key for key, value in p.items()}

In [ ]:
lambda_f = lambdify(f_explicit.subs(subs))

In [ ]:
lambda_f

In [ ]:
output = run(lambda_f, inputs=result.result, **model.ship_parameters, **model.parameters, h=h_)

In [ ]:
output.shape

In [ ]:
def simulate(E, ws, t, us):
    
    simdata = []
    x_=np.deg2rad(np.zeros((6,1)))
    
    for u_,w_ in zip(us,ws):
               
        u = x_[3]
        v = x_[4]
        r = x_[5]
        delta = u_
                        
        x_=x_ + h_*run(lambda_f, u=u, v=v, r=r, delta=delta, **model.ship_parameters, **model.parameters,)
    
        simdata.append(x_.flatten())
        
    simdata = np.array(simdata)
    df = pd.DataFrame(simdata, columns=["x0","y0","psi","u","v","r"], index=t)
    df['delta'] = us
    
    return df

In [ ]:
np.random.seed(42)
df = result.result.iloc[0:100]
N_ = len(df)
E = np.array([[0, 1]]).T
process_noise = 0
ws = process_noise*np.random.normal(size=N_)


us = df['delta'].values
t_ = df.index
h_ = t_[1]-t_[0]

df2 = simulate(E=E, ws=ws, t=t_, us=us)